In [1]:
# xMD testing
import pandas as pd
import os
from xMD.xMD import xMD
from xMD.MD_Settings import GROMACS_Settings

settings = GROMACS_Settings()


/home/alexi/Documents/xMD


/home/alexi/miniconda3/envs/xMD/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:




amber14sb_ff_path = os.path.join(os.getcwd())

# Set the GMXLIB environment variable
os.environ["GMXLIB"] = amber14sb_ff_path



In [3]:
settings = GROMACS_Settings()
settings.suffix = "APO_amber99"
# settings.search = "APO"

print(settings.config)

settings.topology = os.path.join(settings.topology,"BPTI")
print(settings.topology)
# make sure to turn on MPI for HPC 
settings.gmx_mpi_on = False


config
topology/BPTI


In [4]:
# list config dir 

configs = os.listdir(settings.config)

configs = [config if ("_" in config and ".mdp" in config) else None for config in configs]
configs = [config for config in configs if config is not None]

print(sorted(configs))


mdps = ['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp']

file_names = ["5PTI_APO_amber99_H_solv_ions.gro", "5PTI_APO_amber99_H.top", "5PTI_APO_amber99_H.itp","5PTI_APO_amber99_CA.itp"]

['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp', '7_prod_10ps_out.mdp', 'md_short.mdp']


In [5]:
# specify ARGS: -P, -R, -N

for rep in range(1,2+1):

    md = xMD(settings, 'BPTI_genvel5', "5PTI", rep)

    # md.check_args()
    md.create_directory_structure(overwrite=True)
    md.run_experiment(config_files=mdps,topology_files=file_names)
    save_path = md.save_experiment()

Replicate number:  1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel5
Trial directory logs:  logs/MD/5PTI/BPTI_genvel5
Trial directory data:  data/MD/5PTI/BPTI_genvel5
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel5
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel5
Environment variables set:  GMXLIB /home/alexi/Documents/xMD
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel5
Trial directory logs:  logs/MD/5PTI/BPTI_genvel5
Trial directory data:  data/MD/5PTI/BPTI_genvel5
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel5
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel5
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel5
Trial directory logs:  logs/MD/5PTI/BPTI_genvel5
Trial directory data:  data/MD/5PTI/BPTI_genvel5
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel5
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel5
Creating directory:  R_1 Creating directory:  R_2 Creating directory:  R_3 

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx grompp -f config/1_minim.mdp -c data/MD/5PTI/BPTI_genvel5/R_1/5PTI_APO_amber99_H_solv_ions.gro -p data/MD/5PTI/BPTI_genvel5/R_1/5PTI_APO_amber99_H.top -o data/MD/5PTI/BPTI_genvel5/R_1/APO_amber99_5PTI_0.tpr -r data/MD/5PTI/BPTI_genvel5/R_1/5PTI_APO_amber99_H_solv_ions.gro -maxwarn 1 -v

Ignoring obsolete mdp entry 'ns-type'

NOTE 1 [file config/1_minim.mdp]:
  You have set rlist larger than the interaction cut-off, but you also have
  verlet-buffer-tolerance > 0. Will set rlist using
  verlet-buffer-tolerance.


NOTE 2 [file config/1_minim.mdp]:
  Setting tcoupl from 'V-rescale' to 'no'. Temperature coupling does not
  apply to steep.


NOTE 3 [file config/1_minim.mdp]:
  Setting pcoupl from 'Parrinello-Rahman' to 'no'. Pressure coupling does
  not apply to steep.

Generating 1-4 interactions: f

Trajectory number:  1
checking input for internal consistency...

Setting the LD random seed to 1014478331

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

processing coordinates...

double-checking input for internal consistency...

Setting gen_seed to -721457153

Velocities were taken from a Maxwell distribution at 300 K

Reading position restraint coords from data/MD/5PTI/BPTI_genvel5/R_1/APO_amber99_5PTI_0.gro

The center of mass of the position restraint coord's is  2.955  2.975  2.938

The center of mass of the position restraint coo

renumbering atomtypes...
processing index file...
Making dummy/rest group for Acc. not used containing 20445 elements
Making dummy/rest group for Freeze containing 20445 elements
Making dummy/rest group for Energy Mon. containing 20445 elements
Number of degrees of freedom in T-Coupling group System is 41385.00
Making dummy/rest group for User1 containing 20445 elements
Making dummy/rest group for User2 containing 20445 elements
Making dummy/rest group for Compressed X containing 20445 elements
Making dummy/rest group for Or. Res. Fit containing 20445 elements
Making dummy/rest group for QMMM containing 20445 elements
T-Coupling       has 1 element(s): System
Energy Mon.      has 1 element(s): rest
Acc. not used    has 1 element(s): rest
Freeze           has 1 element(s): rest
User1            has 1 element(s): rest
User2            has 1 element(s): rest
VCM              has 1 element(s): System
Compressed X     has 1 element(s): rest
Or. Res. Fit     has 1 element(s): rest
QMMM      

Trajectory number:  2
checking input for internal consistency...

Setting the LD random seed to -954507779

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

processing coordinates...

double-checking input for internal consistency...

Reading position restraint coords from data/MD/5PTI/BPTI_genvel5/R_1/APO_amber99_5PTI_1.gro

The center of mass of the position restraint coord's is  2.957  2.974  2.945

The center of mass of the position restraint coord's is  2.957  2.974  2.945

converting bonded parameters...

initialising group options...

...

Group     0 (         System) has 20445 elements
Group     1 (        Protein) has   892 elements
Group     2 (      Protein-H) has   454 elements
Group     3 (        C-alpha) has    58 elements
Group     4 (       Backbone) has   174 elements
Group     5 (      MainChain) has   233 elements
Group     6 (   MainChain+Cb) has   285 elements
Group     7 (    MainChain+H) has   289 elements
Group     8 (      SideChain) has   603 elements
Group     9 (    SideChain-H) has   221 elements
Group    10 (    Prot-Masses) has   892 elements
Group    11 (    non-Protein) has 19553 elements
Group    12 (          Water) has 19509 elements
Group    13 (            SOL) has 19509 elements
Group    14 (      non-Water) has   936 elements
Group    15 (            Ion) has    44 elements
Group    16 ( Water_and_ions) has 19553 elements
Reading frame       0 time    0.000   
Precision of data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_2-nojump.xtc is 0.001 (nm)

Back Off! I just backed up data/MD/5PTI/

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Running trjconv command 2:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_3-mol.xtc', '-s', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_3.tpr', '-pbc', 'nojump', '-o', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_3-nojump.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 3:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_3-nojump.xtc', '-s', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_3.tpr', '-dump', '0', '-o', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_3-nojump.pdb']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjc

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx trjconv -f data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_5-nojump.xtc -s data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_5.tpr -dump 0 -o data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_5-nojump.pdb

Will write pdb: Protein data bank file
Reading file data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_5.tpr, VERSION 2023.2 (single precision)
Reading file data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_5.tpr, VERSION 2023.2 (single precision)
Group     0 (         System) has 20445 elements
Group     1 (        Protein) has   892 elements
Group     2 (      Protein-H) has   454 elements
Group     3 (        C-alpha) has    58 elements
Group     4 (       Backbone) has   174 elements
Group     5 (      MainChain) has   233 elements
Group     6 (   MainChain+Cb) has   285 elements
Group     7 (    MainCh

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Running trjconv command 2:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_6-mol.xtc', '-s', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_6.tpr', '-pbc', 'nojump', '-o', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_6-nojump.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 3:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_6-nojump.xtc', '-s', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_6.tpr', '-dump', '0', '-o', 'data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_6-nojump.pdb']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Note that ma

Group     0 (         System) has 20445 elements
Group     1 (        Protein) has   892 elements
Group     2 (      Protein-H) has   454 elements
Group     3 (        C-alpha) has    58 elements
Group     4 (       Backbone) has   174 elements
Group     5 (      MainChain) has   233 elements
Group     6 (   MainChain+Cb) has   285 elements
Group     7 (    MainChain+H) has   289 elements
Group     8 (      SideChain) has   603 elements
Group     9 (    SideChain-H) has   221 elements
Group    10 (    Prot-Masses) has   892 elements
Group    11 (    non-Protein) has 19553 elements
Group    12 (          Water) has 19509 elements
Group    13 (            SOL) has 19509 elements
Group    14 (      non-Water) has   936 elements
Group    15 (            Ion) has    44 elements
Group    16 ( Water_and_ions) has 19553 elements
Reading frame       0 time    0.000   
Precision of data/MD/5PTI/BPTI_genvel5/R_2/APO_amber99_5PTI_6-nojump.xtc is 0.001 (nm)

Back Off! I just backed up data/MD/5PTI/

In [6]:
def parse_gromacs_energy_log(filepath):
    """
    Parse a GROMACS log file to extract energy information at each step into a DataFrame.
    """
   # read the file
    with open(filepath, 'r') as f:
        log = f.readlines()
    
    # find the start of the energy table

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            # find:            Step           Time
            if len(split)==2 and split[0] == "Step" and split[1] == "Time":
                start = i
                break
            #find end

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            if len(split)>1 and split[0] == "Statistics":
                end = i
                break

        # extract the table
        table = log[start+1:end]
    # extract the headers
    df = pd.DataFrame()

    return df


log_file_path = "data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log"
# Parse the log file again with the updated function
energy_df_updated = parse_gromacs_energy_log(log_file_path)


FileNotFoundError: [Errno 2] No such file or directory: 'data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log'